## Play with loading in weights to a keras model and then exporting to a PyTorch model

In [1]:
import sys
# point path to genesis repo
sys.path.insert(
    0,
    '/gpfs/commons/home/tchen/al_project/genesis/analysis/splicing'
)

In [2]:
from definitions.generator.splirent_deconv_conv_generator_concat import load_generator_network
from genesis.generator import build_generator, st_sampled_softmax, st_hardmax_softmax
from pathlib import Path
from keras.models import load_model
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [3]:
model_name = 'genesis_splicing_cnn_target_isoform_00_pwm_and_multisample_hek_only_random_regions_50_epochs_harderentropy_generator.h5'
model_save_dir = '/gpfs/commons/groups/knowles_lab/ting/DEN_splicing_pretrained_models/'

full_path = model_save_dir + model_name

In [4]:
generator_model = load_model(filepath=str(full_path), custom_objects={'K': K, 'st_sampled_softmax': st_sampled_softmax, 'st_hardmax_softmax': st_hardmax_softmax}, compile=False)

2024-02-19 16:48:22.388045: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-02-19 16:48:22.397622: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2400000000 Hz
2024-02-19 16:48:22.399684: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557d5c3da380 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-02-19 16:48:22.399713: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Instructions for updating:
Use `tf.random.categorical` instead.



In [ ]:
generator_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence_class_seed (InputLayer (32, 1)              0                                            
__________________________________________________________________________________________________
lambda_rand_sequence_class (Lam (32, 1)              0           sequence_class_seed[0][0]        
__________________________________________________________________________________________________
noise_input_1 (InputLayer)      (32, 100)            0                                            
__________________________________________________________________________________________________
noise_input_2 (InputLayer)      (32, 100)            0                                            
__________________________________________________________________________________________________
lambda_ran

Get order of weights

In [23]:
generator_model.get_config()['layers'][12]

{'name': 'policy_batch_norm_0',
 'class_name': 'BatchNormalization',
 'config': {'name': 'policy_batch_norm_0',
  'trainable': True,
  'axis': -1,
  'momentum': 0.99,
  'epsilon': 0.001,
  'center': True,
  'scale': True,
  'beta_initializer': {'class_name': 'Zeros', 'config': {}},
  'gamma_initializer': {'class_name': 'Ones', 'config': {}},
  'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}},
  'moving_variance_initializer': {'class_name': 'Ones', 'config': {}},
  'beta_regularizer': None,
  'gamma_regularizer': None,
  'beta_constraint': None,
  'gamma_constraint': None},
 'inbound_nodes': [[['policy_deconv_0', 0, 0, {}]],
  [['policy_deconv_0', 1, 0, {}]]]}

In [9]:
len(generator_model.get_weights())

36

Got the weights!

Let's save the weights into a numpy object file

In [8]:
save_path = '/gpfs/commons/groups/knowles_lab/ting/DEN_splicing_generator_weights'
save_name = 'target_isoform_00.npy'

np.save(save_path+save_name, np.array(generator_model.get_weights(), dtype=object), allow_pickle=True)

In [14]:
len(generator_model.get_weights())

36

Now we're gonna try loading in the weights into our PyTorch model

In [2]:
import sys
# point path to our repo
sys.path.insert(
    0,
    '/gpfs/commons/home/tchen/al_project/active-learning-cnns-gps/src'
)

import numpy as np
from models.den import Generator
import torch
from data.old_dataset import create_sequence_templates
from torch import nn

/gpfs/commons/home/tchen/al_project/active-learning-cnns-gps/project-env/lib64/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda')
embedding_template, embedding_mask = create_sequence_templates()

n_samples = 10
n_classes = 1
seq_length = 101
latent_dim = 100
batch_size = 32

In [4]:
torch_generator = Generator(embedding_template=embedding_template,
                                   embedding_mask=embedding_mask,
                                   device=device,
                                   latent_dim=latent_dim,
                                   batch_size=batch_size,
                                   seq_length=seq_length,
                                   n_classes=n_classes,
                                   n_samples=n_samples)

In [5]:
save_path = '/gpfs/commons/groups/knowles_lab/ting/DEN_splicing_generator_weights'
save_name = 'target_isoform_00.npy'

target_isoform_net_weights = np.load(save_path+save_name, allow_pickle=True)

In [6]:
# list of numpy arrays
weights = target_isoform_net_weights.tolist()
for name, param in torch_generator.named_parameters():
    print(name, param.shape)

generator_network.generator_network.0.weight torch.Size([3456, 101])
generator_network.generator_network.0.bias torch.Size([3456])
generator_network.generator_network.1.weight torch.Size([384, 256, 7, 1])
generator_network.generator_network.1.bias torch.Size([256])
generator_network.generator_network.2.weight torch.Size([256])
generator_network.generator_network.2.bias torch.Size([256])
generator_network.generator_network.3.weight torch.Size([256, 192, 8, 1])
generator_network.generator_network.3.bias torch.Size([192])
generator_network.generator_network.4.weight torch.Size([192])
generator_network.generator_network.4.bias torch.Size([192])
generator_network.generator_network.5.weight torch.Size([192, 128, 7, 1])
generator_network.generator_network.5.bias torch.Size([128])
generator_network.generator_network.6.weight torch.Size([128])
generator_network.generator_network.6.bias torch.Size([128])
generator_network.generator_network.7.weight torch.Size([128, 128, 8, 1])
generator_network.

In [47]:
for i, weight in enumerate(weights):
    print(i, weight.shape)

0 (101, 3456)
1 (3456,)
2 (7, 1, 256, 384)
3 (256,)
4 (256,)
5 (256,)
6 (256,)
7 (256,)
8 (8, 1, 192, 256)
9 (192,)
10 (192,)
11 (192,)
12 (192,)
13 (192,)
14 (7, 1, 128, 192)
15 (128,)
16 (128,)
17 (128,)
18 (128,)
19 (128,)
20 (8, 1, 128, 128)
21 (128,)
22 (128,)
23 (128,)
24 (128,)
25 (128,)
26 (8, 1, 128, 64)
27 (64,)
28 (64,)
29 (64,)
30 (64,)
31 (64,)
32 (8, 1, 64, 4)
33 (4,)
34 (1, 436)
35 (1, 436)


In [10]:
torch_generator.generator_network.generator_network

ModuleList(
  (0): Linear(in_features=101, out_features=3456, bias=True)
  (1): ConvTranspose2d(384, 256, kernel_size=(7, 1), stride=(2, 1))
  (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ConvTranspose2d(256, 192, kernel_size=(8, 1), stride=(2, 1))
  (4): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ConvTranspose2d(192, 128, kernel_size=(7, 1), stride=(2, 1))
  (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Conv2d(128, 128, kernel_size=(8, 1), stride=(1, 1), padding=same)
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): Conv2d(128, 64, kernel_size=(8, 1), stride=(1, 1), padding=same)
  (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): Conv2d(64, 4, kernel_size=(8, 1), stride=(1, 1), padding=same)
)

In [7]:
running_keras_weight_idx = 0

for i, layer in enumerate(torch_generator.generator_network.generator_network):
    if isinstance(layer, nn.Linear):
        # transfer linear layer weights and biases
        layer.weight.data = torch.from_numpy(weights[running_keras_weight_idx].T)
        running_keras_weight_idx += 1

        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.ConvTranspose2d):
        # transfer convtranspose2d weights and biases
        layer.weight.data = torch.from_numpy(np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]))
        running_keras_weight_idx += 1
        
        # TODO: double check if we need to reverse biases
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.Conv2d):
        # transfer conv2d weights and biases
        layer.weight.data = torch.from_numpy(np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]))
        running_keras_weight_idx += 1
        
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.BatchNorm2d):
        # transfer batch norm gamma, beta, running mean, running variance
        # order from keras model should be gamma, beta, moving mean, moving variance
        layer.weight.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        layer.running_mean.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        layer.running_var.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1

print(running_keras_weight_idx)


34
